In [1]:
import os
import sys

# from matplotlib import pyplot as plt
# from matplotlib import patches
# from matplotlib import colors
import pandas as pd
import IPython.display as ipd
# import glob
import numpy as np
# import muspy
# import pypianoroll
import torch
from util.play_midi import play_midi
# !pip install miditok symusic
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import DataLoader


c:\Users\joeac\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
config = TokenizerConfig()
tokenizer = REMI()
# midi_files = list(Path("/content/drive/MyDrive/uni/Project/code/audio/midi_songs").glob("**/*.mid"))
midi_files = list(Path("data/jazz_midi").glob("**/*.mid"))
print(f"number of midi_files = {len(midi_files)}")
print(f"number of vocab = {len(tokenizer.vocab)}")

number of midi_files = 934
number of vocab = 282


In [17]:
print(len(midi_files))
file_counter = len(midi_files)
corrupt_files = []
for file in midi_files: # removing corrupted files/ files that cant be fully understood
    try:
        Score(file)
    except:
        # corrupt_files.append(midi_files.pop(file))
        file_holder = file
        midi_files.remove(file)
        corrupt_files.append(file_holder)
print(len(midi_files))
for file in midi_files: # on windows this has to be run 2 times, i have no idea why.... very wierd
    try:
        Score(file)
    except:
        # corrupt_files.append(midi_files.pop(file))
        file_holder = file
        midi_files.remove(file)
        corrupt_files.append(file_holder)
print(len(midi_files))
print(f"corrupt file counter = {len(corrupt_files)} which is {'correct' if file_counter-len(midi_files)==len(corrupt_files) else 'incorrect'}")
file_counter = len(midi_files)

934


914
913
corrupt file counter = 21 which is correct


In [18]:
song = Score(midi_files[-18])
tokenized_song = tokenizer.midi_to_tokens(song)

song1 =tokenizer.tokens_to_midi(tokenized_song)
ids = tokenizer._tokens_to_ids(tokenized_song[-1])
len(tokenizer._ids_to_tokens(ids))
ids

[4,
 4,
 4,
 189,
 236,
 99,
 132,
 197,
 236,
 99,
 132,
 205,
 236,
 99,
 132,
 213,
 240,
 109,
 130,
 219,
 236,
 98,
 126,
 4,
 189,
 236,
 110,
 128,
 230,
 112,
 132,
 243,
 112,
 140,
 193,
 236,
 104,
 126,
 195,
 236,
 106,
 126,
 197,
 232,
 112,
 140,
 230,
 112,
 132,
 236,
 110,
 128,
 201,
 236,
 104,
 128,
 205,
 236,
 110,
 128,
 230,
 112,
 130,
 209,
 236,
 101,
 126,
 211,
 236,
 106,
 126,
 230,
 107,
 126,
 213,
 232,
 112,
 140,
 230,
 112,
 132,
 236,
 110,
 128,
 217,
 236,
 104,
 128,
 4,
 189,
 236,
 110,
 128,
 230,
 112,
 132,
 193,
 236,
 104,
 126,
 195,
 236,
 106,
 126,
 197,
 232,
 112,
 140,
 230,
 112,
 132,
 236,
 110,
 128,
 201,
 236,
 104,
 128,
 205,
 236,
 110,
 128,
 230,
 112,
 130,
 209,
 236,
 101,
 126,
 211,
 236,
 106,
 126,
 230,
 107,
 126,
 213,
 232,
 112,
 140,
 230,
 112,
 132,
 236,
 110,
 128,
 217,
 236,
 104,
 128,
 4,
 189,
 236,
 110,
 128,
 230,
 112,
 132,
 193,
 236,
 104,
 126,
 195,
 236,
 106,
 126,
 197,
 232,
 112,
 1

In [ ]:
# tokenizer.learn_bpe(vocab_size=500, files_paths=midi_files)


In [19]:
from transformers import DataCollatorForLanguageModeling

dataset = DatasetTok( # seq_len = start + seq_len + end
    files_paths=midi_files,
    min_seq_len=50, # 52
    max_seq_len=254, # 514
    tokenizer=tokenizer
)

collator = DataCollator(
    tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"], copy_inputs_as_labels=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

data_loader = DataLoader(dataset=dataset, batch_size=256, collate_fn=collator,)

Loading data: data\jazz_midi: 100%|██████████| 913/913 [00:47<00:00, 19.41it/s]


In [ ]:
# for batch in data_loader:
#     x, y, z = **batch
#     break
# y = **x
# y = tokenizer._ids_to_tokens(x["input_ids"][0].tolist())


In [22]:
from transformers import AutoConfig, GPT2LMHeadModel
from transformers.tokenization_utils_base import BatchEncoding


device = "cuda" if torch.cuda.is_available() else "cpu"
device 


'cpu'

In [23]:
context_length = 256
n_layer = 6
n_head = 8
n_emb = 256 # 512

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_positions=context_length,
    n_layer=n_layer,
    n_head=n_head,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    n_embd=n_emb
)


In [24]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size} parameters")
model

GPT-2 size: 4876800 parameters


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(282, 256)
    (wpe): Embedding(256, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=282, bias=False)
)

In [25]:
model.load_state_dict(torch.load("model/model_weights/remi_gpt2.pt"))

<All keys matched successfully>

In [29]:

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 100
for epoch in range(epochs):
    total_loss = []
    model.train()
    for batch in data_loader:
        model.zero_grad()
        # full = torch.tensor(batch).to(device)
        full_inputs = BatchEncoding()
        full_inputs.update(batch)
        inputs = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        mask = batch["attention_mask"].to(device)
        outputs = model(**full_inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss.append(loss)
    print(f"Epock = {epoch+1} \nloss = {loss}")
# torch.save(model.state_dict(), "remi_gpt2.pt")

Epock = 1 
loss = 4.080315113067627
Epock = 2 
loss = 4.88307523727417
Epock = 3 
loss = 4.390296936035156
Epock = 4 
loss = 4.267981052398682
Epock = 5 
loss = 4.30527925491333
Epock = 6 
loss = 4.03278923034668
Epock = 7 
loss = 3.999056577682495
Epock = 8 
loss = 4.026047706604004
Epock = 9 
loss = 3.9749960899353027
Epock = 10 
loss = 3.8980295658111572
Epock = 11 
loss = 3.866860866546631
Epock = 12 
loss = 3.834446907043457
Epock = 13 
loss = 3.787245035171509
Epock = 14 
loss = 3.7355384826660156
Epock = 15 
loss = 3.699366569519043
Epock = 16 
loss = 3.6793198585510254
Epock = 17 
loss = 3.6651341915130615
Epock = 18 
loss = 3.6434450149536133
Epock = 19 
loss = 3.61387038230896
Epock = 20 
loss = 3.588630437850952
Epock = 21 
loss = 3.5734989643096924
Epock = 22 
loss = 3.558274269104004
Epock = 23 
loss = 3.543020486831665
Epock = 24 
loss = 3.530461311340332
Epock = 25 
loss = 3.512260675430298
Epock = 26 
loss = 3.4939494132995605
Epock = 27 
loss = 3.4759960174560547
Epock

In [26]:
model.eval()

with torch.no_grad():
    for batch in data_loader:
        full_inputs = BatchEncoding()
        full_inputs.update(batch)
        labels = batch["labels"].to(device)
        outputs = model(**full_inputs)
        break
print(outputs)

CausalLMOutputWithCrossAttentions(loss=tensor(3.5528), logits=tensor([[[-7.2206, -1.6329,  0.4266,  ..., -7.0236, -7.1012, -7.2409],
         [-4.8006, -0.9073, -0.6339,  ..., -4.4813, -4.6574, -4.7848],
         [-4.3192, -0.8067, -0.6086,  ..., -4.0559, -4.0786, -4.1680],
         ...,
         [-3.4756, -0.5429, -0.2670,  ..., -4.1743, -4.1017, -3.7221],
         [-2.8764, -0.6399,  5.4614,  ..., -2.8341, -2.5352, -2.9797],
         [-5.2903, -1.5465,  2.3038,  ..., -5.6244, -5.2316, -5.3831]],

        [[-7.2206, -1.6329,  0.4266,  ..., -7.0236, -7.1012, -7.2409],
         [-6.5397, -1.5894, -0.1189,  ..., -7.0216, -6.9668, -6.9418],
         [-4.6869, -1.0314, -2.2262,  ..., -4.7254, -4.7426, -4.8359],
         ...,
         [-5.2193, -0.5442, -0.3252,  ..., -5.9171, -6.2553, -5.7020],
         [-3.5188, -0.7338,  5.4909,  ..., -3.4359, -3.1686, -3.5984],
         [-4.5505, -1.2094,  0.5431,  ..., -4.6970, -4.4378, -4.5178]],

        [[-7.2206, -1.6329,  0.4266,  ..., -7.0236, -7

In [31]:

output_tokens = torch.tensor([[0]*256]*256) # [[0]*context_length]*batch_size
for i in range(len(outputs["logits"])):
    output_tokens[i] = torch.argmax(outputs["logits"][i], dim=-1)

tokens_output = tokenizer._ids_to_tokens(output_tokens[0].tolist())
# print(output_tokens[0])

In [33]:
midi_song_output = tokenizer.tokens_to_midi([tokens_output])
output_file_path = Path("data/output/", "decoded_midi.mid")
midi_song_output.dump_midi(output_file_path)
play_midi(output_file_path)